### Programming for Science and Finance

*Prof. Götz Pfeiffer, School of Mathematical and Statistical Sciences, University of Galway*

# Notebook 8: Programming With Functions II

This notebook accompanies **Part II**. You will:

* learn how to approximate **derivatives** using finite differences and understand these formulas as convolutions
* implement the **trapezoidal rule** and relate it to cumulative sums
* use discrete convolution and smoothing kernels to interpret **numerical differentiation and integration**
* apply and compare **root-finding** methods, including bisection and Newton’s method
* structure numerical algorithms using **classes** to encapsulate solver behavior
* solve simple **ordinary differential equations** with Euler’s method
* develop a unified view of numerical methods in terms of discrete calculus and iterative schemes

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

## Task 1.  Numerical Differentiation

#### Computing the Derivative of a Function

* Recall the **definition of the derivative** of a function $f(x)$:
$$
f'(x) = \lim_{h \rightarrow 0} \frac{f(x+h) - f(x)}{h}.
$$
* Thus, for sufficiently small $h$, we get the following approximation of the derivative:
$$
f'(x) \approx \frac{f(x+h) - f(x)}{h}.
$$
* While it is not entirely clear what "sufficiently small" means, this is something we can implement in Python.
* As a function `diff` with the following parameters:  
  1. the function $f(x)$ that we wish to differentiate,
  2. the value of $h$,    
  returning a Python function that computes an **approximation** of the derivative $f'(x)$.

In [ ]:
def diff(f, h):
    def f1(x):
        return (f(x+h) - f(x))/h
    return f1

* Warning: if $h$ is too small, floating point rounding errors may dominate the result.
* Application to $f(x) = \frac12 x^2$ with derivative $f'(x) = x$ (and $h = 0.01$):

In [ ]:
f = lambda x : x**2/2
f1 = diff(f, 1/100)
[f1(x) for x in range(10)]

* Using a `lambda` expression, the python code for `diff` can be shortened to:

In [ ]:
def diff1(f, h):
    return lambda x: (f(x+h) - f(x))/h

In [ ]:
f = lambda x : x**2/2
f1 = diff1(f, 1/100)
[f1(x) for x in range(10)]

* Alternatively, the formula
  $$
  f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x-h)}{2h}
  $$
  can be used to approximate the derivative of $f$ as
  $$
  f'(x) \approx \frac{f(x+h) - f(x-h)}{2h}
  $$
  for "small" values of `h`.

* This can be implemented in a similar way:

In [ ]:
def diff2(f, h):
    return lambda x: (f(x+h) - f(x-h))/(2*h)

In [ ]:
f = lambda x : x**2/2
f1 = diff2(f, 1/100)
[f1(x) for x in range(10)]

---
**Exercises.**

1. Write a function `diff3` that uses backward differences $(f(x) - f(x-h))/h$ to approximate
   the derivative of $f(x)$.

2. Plot and compare the effects of `diff1`, `diff2` and `diff3` on $f(x) = \frac12x^2$.

---

## Task 2.  Differentiation as Convolution

* In a vectorized version, we often work with a function $f$ in the form of
  a **samples list** `yyy` of $N+1$ function values $y_k = f(x_k)$ for $k = 0, \dots, N$,
  where the $x_k$, $k = 0, \dots, N$ form a list `xxx` often constructed
  as
  ```python
  xxx = np.linspace(a, b, N+1)
  ```
  (Then the function can be plotted easily as `plt.plot(xxx, yyy)`, where `yyy = f(xxx)`.)  
* In such a context
  it is natural to set $h = (b-a)/N = x_{k+1} - x_k$, and the (second) formula for the 
  derivative of $f$ becomes
  $$
  f'(x_k) \approx \frac1{2h}(1 \cdot y_{k+1} + 0 \cdot y_k + (-1) \cdot y_{k-1}).
  $$

* Recall the convolution product $c = a * b$ of lists $a$ and $b$, where the $k$-th element of the product list $c$ is
  $$
  c_k = \sum_{i+j=k} a_i b_j
  = a_{-1} b_{k+1} + a_0 b_k + a_1 b_{k-1} + \dotsm.
  $$

* On comparison with the previous formula, one sees that **differentiation is convolution** of the samples list `yyy` with a kernel of the form $\frac1{2h}[1, 0 ,-1]$.  And note that $[1, 0, -1] = [1, 1] * [1, -1]$ is itself a convolution of a smoothing kernel $s = [1,1]$ and a differentiation kernel $d = [1, -1]$.  (In terms of polynomials, think: $(1 + x) (1 - x) = 1 - x^2$.)

* This yields a python function `diff_samples2` as follows.

In [ ]:
def diff_samples2(yyy, h):
    return np.convolve(yyy, np.array([1, 0, -1])/(2*h), "same")

* Apply this to $f(x) = \frac12 x^2$.

In [ ]:
f = lambda x: x**2/2
xxx = np.linspace(1,10, 11)
yyy = f(xxx)
diff_samples2(yyy, 1)

* Not too bad, except for the endpoints.

* The first differentiation formula could be implemented as follows:

In [ ]:
def diff_samples1(yyy, h):
    return np.convolve(yyy, np.array([1, -1])/h, "same")

In [ ]:
diff_samples1(yyy, 1)

* For more flexibility, we can provide the width `w` of the smoothing kernel as optional parameter, while also fixing the two ends of the interval.

In [ ]:
def diff_samples(yyy, h, w=2):
    ker = np.convolve(np.ones(w), np.array([1, -1]))
    yy1 = np.convolve(yyy, ker/(w*h), "same")
    e = (w+1)//2   # replicate nearest valid value
    return np.pad(yy1[e:-e], e, "edge")


* Padding with `"edge"` replicates nearest valid values to avoid artificial oscillations at the boundaries.
* Let's apply this to $f(x) = \sin(2 \pi x) + \frac15 \cos(10 \pi x)$ for $x \in [0, 1]$.

In [ ]:
f = lambda x: np.sin(2*np.pi*x) + 0.2*np.cos(10*np.pi*x)
a, b = 0, 1
N = 500
xxx = np.linspace(a, b, N+1)
h = (b - a) / N

* We plot $10f(x)$ for the scale, and the approximated $f'(x)$ for 4 different values of `w`.

In [ ]:
yyy = f(xxx)
plt.plot(xxx, 10*yyy, label="10f")

for w in range(1, 5):
    plt.plot(xxx, diff_samples(yyy, h, w), label=f"w={w}")
plt.legend()

---
**Exercises.**

1. Implement backward differences $f'(x) \approx (f(x) - f(x-h))/h$ as `diff_samples3` using a suitable convolution product.

2. Apply `diff_samples3` to $f(x) = \sin(2 \pi x) + \frac15 \cos(10 \pi x)$ for $x \in [0, 1]$,
   plot the result and compare it with the result of `diff_samples`.


---

## Task 3.  Numerical Integration

* The **Riemann integral** $I = \int_a^b f(t)\, dt$ of a function $f$ over the interval $[a, b]$ is defined as a certain limit of weighted sums of function values.

* We can use that weighted sum as approximation to the integral:
  $$
  I \approx h \sum_{k=0}^N f(x_k),
  $$
  where, as above, $[a, b]$ is subdivided in $N$ intervals of length $h = \frac{b-a}{N}$.

* Alternatively, we can use the **Trapezoidal Rule**:
  $$
  I \approx \frac{b-a}{2N} (f(x_0) + 2f(x_1) + \dotsm + 2 f(x_{N-1}) + f(x_N))
  $$


* With this a function `trapez(f, N)` that takes a function `f` and a number `N` as input and constructs a function `F(a, b)` that can compute an approximation to the definite integral $\int_a^b f(t)\, dt$ can be implemented as follows.

In [ ]:
def trapez(f, N):
    def F(a, b):
        xxx = np.linspace(a, b, N+1)
        yyy = f(xxx)
        return (b-a)/(2*N) * (yyy[0] + yyy[-1] + 2*yyy[1:-1].sum())
    return F

* **Example.** $f(x) = 2x$ with $\int_a^b f(t)\, dt = b^2 - a^2$ and $N = 10$.
* Warning: choosing $N$ too small could result in a too coarse approximation.

In [ ]:
f = lambda x : 2*x
F = trapez(f, 10)
F(0, 5)

### Integration as Convolution

* Inspecting the formulas for integration one sees that **integration is convolution** of the samples list `yyy` with a kernel of the form $h [1, 1, \dots, 1]$, or of the form $\frac{h}{2}[1, 2, 2, \dots, 2, 1]$.  

* Note that $[1, 2, 2, \dots, 2, 1] = [1, 1] * [1, 1, \dots, 1]$, the convolution of a **short smoothing kernel** $[1,1]$ of width $w = 2$  and a **long summation kernel** $[1,1,\dots, 1]$(think: $(1 + x)(1 + x + x^2) = (1 + 2x + 2x^2 + x^3)$).

* Convolution with the long summation kernel $[1, 1, \dots, 1]$ is actually more efficiently
computed as a **cumulative sum**. That's what `np.cumsum()` does.

In [ ]:
l = [1, 2, 3, 4]
print(np.convolve(l, [1,1,1,1])[:len(l)])
print(np.cumsum(l))

* With these considerations in mind, we can implement integration by convolution
  with the width $w$ of the short smoothing kernel as an optional parameter:

In [ ]:
def integrate_samples(yyy, h, w=2):
    return np.cumsum((h/w) * np.convolve(yyy, np.ones(w)))[:len(yyy)]

* Note how we need to truncate the result to make it the same length as the samples array `yyy`. 
* Let's apply this to $f(x) = \sin(2 \pi x) + \frac15 \cos(10 \pi x)$ for $x \in [0, 1]$.

In [ ]:
f = lambda x: np.sin(2*np.pi*x) + 0.2*np.cos(10*np.pi*x)

a, b, N = 0, 1, 500
h = (b - a) / N

xxx = np.linspace(a, b, N+1)
yyy = f(xxx)

* Compute the antiderivative and plot the result.

In [ ]:
zzz = integrate_samples(yyy, h)
plt.plot(xxx, zzz, label="F")
plt.legend()

* Now differentiate the antiderivative and compare the result with the original function $f$.

In [ ]:
plt.plot(xxx, diff_samples(zzz, h), label="F'(x)")
plt.plot(xxx, yyy, label="f(x)")
_ = plt.legend()

* Indistinguishable!  That's exactly what the **Fundamental Theorem of Calculus** (FTC) predicts.

* Looking at differentiation and integration as convolution, FTC takes this form: 
  $$
  [1,-1] * [1, 1, 1, \dotsc] = [1],
  $$
  i.e., the short difference kernel $[1, -1]$ and the infinite summation kernel $[1,1,1,\dots]$
  are each other's **convolution inverses**
  (think geometric series: $\frac{1}{1-x} = \sum_i x^i$).

---
**Exercises.**

1. Write a Python function `antiderivative(f, a, N)` that uses `trapez(f, N)` to approximate the antiderivative $F(x) = \int_a^x f(t)\,dt$ of $f(x)$.

2. Write a Python function `trapezoid_samples(yyy, h)` that first convolves `yyy` with
   the short smoothing kernel $k = \frac{h}2 [1, 1]$ and then returns the
   cumulative sum of the results as the list of antiderivative samples.  Compare the output
   with `integrate_samples(yyy, h)` for a test function.

3. Sample the function $f(x) = \sin(10x)+\frac{2}{5}\cos(20x)$ on the interval $x \in [0, \frac{\pi}{5}]$ using $N = 20$ subintervals.
  Compute the samples `yyy = f(xxx)` and the step size $h$.

4. Using `integrate_samples`, compute approximations to the antiderivative
   corresponding to window sizes $w = 1, 2, 3$.  Compute the exact antiderivative $F(x)$
   with $F(0) = 0$.  Plot all four curves, and discuss how they differ and why.

---

## Task 4.  Root Finding

A **root** (or **zero**) of a function $f \colon \mathbb{R} \to \mathbb{R}$
is a number $x \in \mathbb{R}$ such that $f(x) = 0$.  
Often, the equation $f(x) = 0$ cannot be solved for $x$.

A **Root Finding Algorithm** can approximate a root of $f$, without a guarantee to find **all** roots of $f$.  
There are several such algorithms, with varying degrees of sophistication.
We discuss two simple strategies.

### Bisection

For a given **tolerance** $\epsilon$, and a maximum number $M$ of iterations,

* **Initially**, pick an interval $[a, b]$ such that $f(a) \cdot f(b) < 0$, i.e., $f(a)$ and $f(b)$ have **opposite signs**.  (Then by the [IVT](https://en.wikipedia.org/wiki/Intermediate_value_theorem), there exists at least one $x \in (a, b)$ with $f(x) = 0$.)

* **Repeat** at most $M$ times:
  * $c = \frac{a + b}{2}$ (midpoint)
  * replace either $a$ or $b$ by $c$ so that $f(a) \cdot f(b) < 0$

  until $|f(c)| < \epsilon$  
* **Return** $c$ as an approximation to a root of $f$

This method is simple but slow, as it needs approximately $\log_2 \frac{b-a}{\epsilon}$ steps.

### Newton's Method

The [Newton-Raphson Method](https://en.wikipedia.org/wiki/Newton%27s_method) uses the derivative $f'(x)$ to locate a zero of $f(x)$. 

For a given **tolerance** $\epsilon$, and a maximum number $M$ of iterations,

* **Initially**, guess a value $x$.

* **Repeat** at most $M$ times:
  * replace $x$ by $x - f(x)/f'(x)$

  until $|f(x)| < \epsilon$

* **Return** $x$ as an approximation to a root of $f$.

This method converges quadratically, if the initial guess $x$ is close enough to the actual root.

---
**Exercises.**

1. Implement the bisection method as a Python function `bisection`.

2. Apply your `bisection` function to solve $x^2 = 2$.

1. Implement the Newton method as a Python function `newton`.

2. Apply your `newton` function to solve $x^2 = 3$.

---

## Task 5. A Root Finding Class

Seeing the above two root finding methods have a lot of structure in common, it might be a good idea to implement both (and perhaps other root finding methods) with a common API.

Both methods work with more or less the same setup, consisting of:

* a function $f(x)$,
* its derivative $f'(x)$ (optional)
* a tolerance $\epsilon$ (default value $10^{-6}$)
* a maximal number $M$ of iterations (default value $99$).

With this setup, one can build a function `solve` that finds a root, starting with one or two initial values.

For an implementation, we choose the following **design**. We define an abstract parent **class**
`RootFinder` for the general setup.  Different strategies can then be defined as **subclasses** of `RootFinder` and only need to implement the `solve` function.

In [ ]:
class RootFinder:
    def __init__(self, f, f1=None, eps=1e-6, M=99):
        self.f = f
        self.f1 = f1
        self.eps = eps
        self.M = M

    def solve(self, a, b= None):
        raise NotImplementedError("Subclass must implement solve().")

The `solve` method in the parent class serves only as a placeholder.|

The Bisection method as a subclass `BisectionSolver` inherits the `__init__` method and
hence needs no implementatoin of its own.

In [ ]:
class BisectionSolver(RootFinder):
    def solve(self, a, b):  # overwrite parent's method
        f = self.f
        assert f(a) * f(b) < 0, "end points must have opposite signs"
        for i in range(self.M):
            c = (a+b)/2
            if abs(f(c)) < self.eps:
                return c
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
        print("Warning: maximum iterations reached.")
        return c  # now i >= M


Application to $x^2 = 2$, i.e., find $\sqrt{2}$:

* $f(x) = x^2 - 2$.
* $[a, b] = [1, 2]$.

In [ ]:
solver = BisectionSolver(lambda x: x**2 - 2)
x = solver.solve(1, 2)
print(x, x**2)

The Newton method as a subclass `NewtonSolver` inherits the `__init__` method and
hence needs no implementatoin of its own.

In [ ]:
class NewtonSolver(RootFinder):
    def solve(self, a, b=None):
        f, f1 = self.f, self.f1
        assert f1 is not None, "f1 needed"
        for i in range(self.M):
            y = f(a)
            if abs(y) < self.eps:
                return a
            a -= y/f1(a)
        print("Warning: maximum iterations reached.")
        return a  # now i >= M

Application to $x^2 = 3$, i.e., find $\sqrt{3}$:

* $f(x) = x^2 - 3$
* $f'(x) = 2x$
* $a = 1$.

In [ ]:
solver = NewtonSolver(lambda x: x**2 - 3, lambda x: 2*x)
x = solver.solve(1)
print(x, x**2)

---
**Exercises.**
1. Find a root of $f(x) = x^3 - x - 2$ between $1$ and $2$.

1. Solve $\cos x = x$ (with $[a, b] = [0, 1]$).

1. Solve $x e^x = 2$ (with $[a, b] = [0, 1]$).

3. Implement a Newton solver that computes $f'(x)$ from $f(x)$ rather than using an explict argument.

4. Modify the solver classes by renaming `solve` to `__call__`.  Why would that be a good idea?

---

## Task 6. Solving Differential Equations

A **first order initial value problem** (IVP)  has the form
$$
y'(t) = f(t, y(t)),\\
y(t_0) = y_0,
$$
i.e., a first order **differential equation** (expressing the derivative of a function $y(t)$ in terms of $y$ and $t$) and an **initial condition** (specifying the value of $y$ at a particular point).   The goal is to find a function $y(t)$ that satisfies both the differential equation and the initial condition.

### Euler's Method

A simple method for approximating a solution to an IVP uses the approximation to the derivative
$$
y'(t) \approx \frac{y(t + h) - y(t)}{h}
$$
Setting
$$
t_n = t_{n-1} + h,\quad n > 0,
$$
and
$$
y_n = y(t_n),
$$
we can rewrite
$$
f(t_0, y_0) = y'(t_0) \approx \frac{y(t_1) - y(t_0)}{h}
= \frac{y_1 - y_0}{h}
$$
as an approximation
$$
y_1 \approx y_0 + h \cdot f(t_0, y_0)
$$
and then, continuing in that fashion:
$$
y_{n+1} \approx y_n + h \cdot f(t_n, y_n).
$$

This process can be implemented as follows.

In [ ]:
def euler(f, t0, y0, h):
    def approx(n):
        t, y = t0, y0
        for i in range(n):
            y += h * f(t, y)
            t += h
        return y
    return approx

**Example.** Consider
$$
y' = 2y +t^2 - t \\ y(0) = 1.
$$
with $h = 0.01$, say.

An exact solution to this IVP is $y = e^{2t} - t^2/2$, as can be easily verified:
clearly, $y(0) = 1$, and $y' = 2e^{2t} - t$.


In [ ]:
e = euler(lambda t, y: 2*y + t**2 - t, 0, 1, 0.01)
e(100)

Plot the estimate and the exact solution:

In [ ]:
ttt = np.linspace(0, 1, 101)
y = lambda t: np.exp(2*t) - t**2/2  # exact solution
plt.plot(ttt, y(ttt), label="exact")
plt.plot(ttt, [e(n) for n in range(101)], label="estimate")
_ = plt.legend()


---
**Exercises.**
1. Solve $y' = -2y$, $y(0) = 100$ (with $h = 1$ and $N = 100$) and compare the result with the exact solution $y(t) = 100 e^{-2t}$.
---

## Summary

* introduced **finite difference methods** for numerical differentiation
* interpreted differentiation as **convolution with short kernels**
* implemented **numerical integration**, including the trapezoidal rule
* connected integration to **cumulative sums and convolution**
* applied two core **root-finding** methods: bisection and Newton’s method
* organized solvers into **classes** to encapsulate algorithms
* solved simple **ordinary differential equations** using Euler’s method
* highlighted how differentiation, integration, and iteration form the basis of numerical computation
